# Test the NCF module under folder [cf_ec2](../cf_ec2) with ml-1m dataset, save the best model (using integrated modules with compile and fit components, with gmf and mlp pretrain)

#### 4/18/2020

In [1]:
import numpy as np 
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import (
    Adam,
    Adamax,
    Adagrad,
    SGD,
    RMSprop
)
from tensorflow.keras.layers import (
    Embedding, 
    Input,
    Flatten, 
    Multiply, 
    Concatenate,
    Dense
)

import sys
sys.path.append('../')
from cf_ec2 import (
    GMF,
    MLP,
    NCF,
    Data,
    evaluation,
    evaluation_grouped
)

## step 1: load the data

In [2]:
train = pd.read_csv('../data/ml-1m-all-train.csv',sep=',',header=0,names=['user','item','rating','event_ts'])
test = pd.read_csv('../data/ml-1m-all-test.csv',sep=',',header=0,names=['user','item','rating','event_ts'])

In [3]:
test.user.nunique(), test.shape

(6040, (250088, 4))

In [4]:
train.user.nunique(), train.shape

(6040, (750121, 4))

In [5]:
train.head(3)

,user,item,rating,event_ts
0,1,3186,4.0,978300019
1,1,1721,4.0,978300055
2,1,1270,5.0,978300055


In [6]:
test.head(3)

,user,item,rating,event_ts
0,1,1545,4.0,978824139
1,1,527,5.0,978824195
2,1,745,3.0,978824268


## step 2: prepare the data for gmf model training

In [7]:
%%time

dataset = Data(
    train=train,
    test=test,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_time='event_ts',
    binary=True,
    n_neg=4,
    n_neg_test=100,
    n_neg_limit=0
)
dataset.prepTrainDNN(negSample=True)
dataset.prepTestDNN(group=False)

CPU times: user 1min 13s, sys: 1.96 s, total: 1min 15s
Wall time: 1min 16s


In [8]:
dataset.interaction_train.head(3)

,user,item_interacted,item_negative
0,0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5..."
1,1,"{7, 11, 16, 17, 20, 28, 36, 40, 41, 42, 43, 44...","{0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15..."
2,2,"{130, 131, 132, 133, 134, 135, 136, 9, 137, 13...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14..."


#### prepare the test dataset

In [9]:
newItems = set(dataset.items_test)-set(dataset.items)
idx2del = []
for idx,item in enumerate(dataset.items_test):
    if item in newItems:
        idx2del.append(idx)

length_test_original = len(dataset.users_test)
dataset.users_test = [
    dataset.users_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.items_test = [
    dataset.items_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.ratings_test = [
    dataset.ratings_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]

## step 3: create the model architecture and fit model with training data

In [10]:
train.user.nunique(), train.item.nunique()

(6040, 3660)

In [11]:
n_users = 6040
n_items = 3660
# n_factors_gmf = 8
# layers_mlp = [64,32,16,8]
n_factors_gmf = 4
layers_mlp = [16,8,4]
reg_gmf = 0.
reg_layers_mlp = [0.,0.,0.,0.]
learning_rate = 0.001
flg_pretrain = ''
filepath = ''
filepath_mlp_pretrain = ''
filepath_mlp_pretrain = ''
num_epochs = 20
batch_size = 256

ncf = NCF(
    n_users=n_users,
    n_items=n_items,
    n_factors_gmf=n_factors_gmf,
    layers_mlp=layers_mlp,
    reg_gmf=reg_gmf,
    reg_layers_mlp=reg_layers_mlp
)
ncf.create_model()

In [12]:
ncf.compile(learning_rate=learning_rate)

In [13]:
ncf.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_mlp_User (Embedding)  (None, 1, 8)         48320       user_input[0][0]                 
__________________________________________________________________________________________________
embedding_mlp_Item (Embedding)  (None, 1, 8)         29280       item_input[0][0]                 
______________________________________________________________________________________________

In [14]:
hist = ncf.fit(
    dataset=dataset,
    batch_size=batch_size,
    num_epochs=num_epochs,
    path_model_weights='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3/ncf-weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5',
    path_csvlog='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3/ncf_log.csv'
)

Train on 3750605 samples, validate on 25252525 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 0.16500, saving model to /Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3/ncf-weights-improvement-01-0.1650.hdf5
3750605/3750605 - 106s - loss: 0.3394 - accuracy: 0.8484 - val_loss: 0.1650 - val_accuracy: 0.9430
Epoch 2/20

Epoch 00002: val_loss improved from 0.16500 to 0.15292, saving model to /Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3/ncf-weights-improvement-02-0.1529.hdf5
3750605/3750605 - 101s - loss: 0.2934 - accuracy: 0.8704 - val_loss: 0.1529 - val_accuracy: 0.9448
Epoch 3/20

Epoch 00003: val_loss improved from 0.15292 to 0.14427, saving model to /Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3/ncf-weights-improvement-03-0.1443.hdf5
3750605/3750605 - 106s - loss: 0.2822 - accuracy: 0.8757 - val_loss: 0.1443 - val_accuracy: 0.9477
Epoch 4/20

Epoch 00004: val_loss did not improve from 0.14427
3750605/3750605 - 103s - loss: 0.2771 - 

#### try to load the parameters from the best model

In [15]:
ncf.model.load_weights('../metadata/ncf3/ncf-weights-improvement-14-0.1430.hdf5')

In [16]:
scores = ncf.model.evaluate(
    x = [
        np.array(dataset.users_test),
        np.array(dataset.items_test)
    ],
    y = np.array(dataset.ratings_test),
    verbose=0
)

In [17]:
scores

[0.1429816370327044, 0.94643754]

In [18]:
ncf.model.save('../metadata/ncf3/ncf-best.hdf5')

In [19]:
model3 = keras.models.load_model('../metadata/ncf3/ncf-best.hdf5')

In [20]:
%%time 

evaluator = evaluation_grouped.metricsEval(
    model=model3,
    users=dataset.users,
    items=dataset.items
)
evaluator.getRecs()
rmse,auc,logloss = evaluator.getOverlapBasedMetrics(
    dataset.users_test,
    dataset.items_test,
    dataset.ratings_test
)
rmse,auc,logloss

100%|██████████| 6040/6040 [09:17<00:00, 10.83it/s]


CPU times: user 21min 32s, sys: 2min 41s, total: 24min 14s
Wall time: 17min 47s


(0.20243901093434202, 0.8561481331307735, 0.1429816375232299)

In [21]:
recall,precision,ndcg,map2 = evaluator.getRankBasedMetrics(
    dataset.users_test,
    dataset.items_test,
    dataset.ratings_test
)
recall,precision,ndcg,map2

(0.02646146203922253,
 0.04362582781456954,
 0.044529338374450375,
 0.008927550968871921)

#### keep training the model

In [23]:
hist = ncf.fit(
    dataset=dataset,
    batch_size=batch_size,
    num_epochs=num_epochs,
    path_model_weights='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3_2/ncf-weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5',
    path_csvlog='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3_2/ncf_log.csv'
)

Train on 3750605 samples, validate on 25252525 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 0.14317, saving model to /Users/xyin/Documents/work/projects/rec_utils/metadata/ncf3_2/ncf-weights-improvement-01-0.1432.hdf5
3750605/3750605 - 100s - loss: 0.2546 - accuracy: 0.8897 - val_loss: 0.1432 - val_accuracy: 0.9458
Epoch 2/20

Epoch 00002: val_loss did not improve from 0.14317
3750605/3750605 - 100s - loss: 0.2544 - accuracy: 0.8899 - val_loss: 0.1462 - val_accuracy: 0.9442
Epoch 3/20

Epoch 00003: val_loss did not improve from 0.14317
3750605/3750605 - 100s - loss: 0.2542 - accuracy: 0.8900 - val_loss: 0.1445 - val_accuracy: 0.9448
Epoch 4/20

Epoch 00004: val_loss did not improve from 0.14317
3750605/3750605 - 99s - loss: 0.2541 - accuracy: 0.8900 - val_loss: 0.1482 - val_accuracy: 0.9434
Epoch 5/20

Epoch 00005: val_loss did not improve from 0.14317

Epoch 00005: ReduceLROnPlateau reducing learning rate to 2.700000040931627e-05.
3750605/3750605 - 99s - loss: 0.2540